# Proyecto AirBNB

### Estudio del Contexto

#### ¿Qué es AirBNB?

Antes de hacer la exploración y el análisi de los datos, nos ponemos en contexto.

"Airbnb es una compañía que ofrece una plataforma digital dedicada a la oferta de alojamientos a particulares y turísticos (alquiler vacacional) mediante la cual los anfitriones pueden publicitar y contratar el arriendo de sus propiedades con sus huéspedes; anfitriones y huéspedes pueden valorarse mutuamente, como referencia para futuros usuarios." Wikipedia [es.wikipedia.org/wiki/Airbnb](https://es.wikipedia.org/wiki/Airbnb)

De esta manera sabemos qué vamos a encontrar, información de alojamientos y referencias.

#### Exploramos el directorio Datasets

```` javascript
ls -s datasets
total 627988
 432868 calendar.csv
 86000 listings.csv
 109120 reviews.csv
````

Vemos que son Archivos tipo csv de tamaño medio a grande, por lo tanto es lo primero que vamos a tener en cuenta.

Pasamos a hacer el EDA

##### Analizamos el archivo ````calendar.csv````

````
listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
50778,2020-04-26,f,"$2,655.00","$2,655.00",5,1125
133654,2020-04-27,t,"$1,150.00","$1,150.00",4,1125
133654,2020-04-28,t,"$1,150.00","$1,150.00",4,1125
133654,2020-04-29,t,"$1,150.00","$1,150.00",4,1125
133654,2020-04-30,t,"$1,150.00","$1,150.00",4,1125
133654,2020-05-01,t,"$1,150.00","$1,150.00",4,1125
133654,2020-05-02,t,"$1,150.00","$1,150.00",4,1125
````

A simple vista, parece que es una tabla de hechos, desde el punto de vista de datawarehouse, con los siguientes campos

* **listing_id**: Es el id de la tabla  que detalla los alojamientos, el lugar físico
* **date**: fecha
* **available**: **t** es True, **f** is false
* **price**: precio por noche
* **adjusted_price**: 
* **minimum_nights**: cantidad minima de noches
* **maximum_nights**: cantidad maxima de noches

Después de hacer un insight en el archivo listings, verificamos que calendat asocia un alojamiento con una fecha, disponibilidad y precio, siendo los dos últimos campos, redundantes


Procedemos a realizar la limpieza

In [5]:
import pandas as pd
import numpy as np

In [ ]:
calendar = pd.read_csv('../datasets/calendar.csv', sep=',', usecols=['listing_id', 'date', 'available', 'price', 'adjusted_price'])

calendar.listing_id = calendar.listing_id.astype('uint16').copy()

calendar.available = calendar.available.apply(lambda x: False if x == 'f' else True).copy()

calendar.date = pd.to_datetime(calendar.date).copy()
calendar.price = calendar.price.apply(lambda x: x.replace("\"", "").replace("$", "").replace(",", "")).copy()
calendar.adjusted_price = calendar.adjusted_price.apply(lambda x: x.replace("\"", "").replace("$", "").replace(",", "")).copy()
calendar.price = calendar.price.astype("float32").copy()
calendar.adjusted_price = calendar.adjusted_price.astype("float32").copy()


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
import pymysql
import pandas as pd
import numpy as np

HOST='localhost'
SCHEMA='airbnb'
USER='root'
PORT=3306
# PASS='****'


# calendar_limpio = pd.read_csv('../datasets/calendar_limpio.csv', sep="\t")


from sqlalchemy import create_engine
# cnx = create_engine('mysql+pymysql://[user]:[pass]@[host]:[port]/[schema]', echo=False)

# cnx = create_engine(f'mysql+pymysql://${USER}:${PASS}@{HOST}:{PORT}/{SCHEMA}', echo=False)
cnx = create_engine(f'mysql+pymysql://root:{PASS}@{HOST}:{PORT}/{SCHEMA}', echo=False)

# calendar_limpio.to_sql(name="calendar", con=cnx)

## Archivo **listings.csv**

Procederemos a leer via DataFrame el archivo **listings.csv**

In [6]:
listings = pd.read_csv('../datasets/listings.csv', low_memory=False)

In [7]:
listings.head()
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23729 entries, 0 to 23728
Columns: 106 entries, id to reviews_per_month
dtypes: float64(23), int64(21), object(62)
memory usage: 19.2+ MB


Vemos que contiene 106 columnas!!!

````[id, ...., calculated_host_listings_count_shared_rooms, reviews_per_month]````

In [ ]:
colunmas_listings = listings.columns

for i, c in enumerate(colunmas_listings):
    print(c, end=", ")

Luego de un análisis se puede ver como tablas relacionales condensadas en una sola, después de un análisis de las columnas llegamos a la conclusión que podemos extraer al menos 8 tablas según la naturaleza de las columnas.

Separamos por ejemplo, la información de los anfitriones en una tabla llamada **hosts** y la almacenamos en **MySQL**.

In [15]:
columnas_host = ['host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified']

host = listings[columnas_host]
host = host.drop_duplicates()

host.to_sql(name="hosts", con=cnx, if_exists='replace')
# host.to_csv('../datasets/hosts.csv', encoding='utf-8', sep=';')


15536

Actualizamos la tabla quitando la info de los anfitriones en el dataframe de listings

In [16]:
listings_filtered = listings.drop(columnas_host[1:], axis=1)

Usando este mismo formato continuamos con las siguientes tablas que podemos generar

In [17]:
columnas_info1 = ['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url']
columnas_info2 = ["id","street", "neighbourhood_cleansed", "neighbourhood_group_cleansed","require_guest_profile_picture","require_guest_phone_verification",
           "calculated_host_listings_count", "calculated_host_listings_count_entire_homes", "calculated_host_listings_count_shared_rooms",
           "calculated_host_listings_count_private_rooms"]
columnas_location = ['id', 'neighbourhood', 'city', 'state',
       'zipcode', 'market', 'smart_location', 'country_code', 'country',
       'latitude', 'longitude', 'is_location_exact']
columnas_propiedad = ['id','property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'bed_type', 'amenities', 'square_feet']
columnas_precio = ['id','price', 'weekly_price', 'monthly_price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people']
columnas_disponibilidad = ['id', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped']
columnas_review = ['id','number_of_reviews', 'number_of_reviews_ltm', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'reviews_per_month']

In [18]:
info1 = listings[columnas_info1]
info2 = listings[columnas_info2]
location = listings[columnas_location]
datos_propiedad = listings[columnas_propiedad]
precios = listings[columnas_precio]
disponibilidad = listings[columnas_disponibilidad]
reviews_info = listings[columnas_review]

In [19]:
listings_filtered = listings_filtered.drop(columnas_info1[1:], axis=1)
listings_filtered = listings_filtered.drop(columnas_info2[1:], axis=1)
listings_filtered = listings_filtered.drop(columnas_location[1:], axis=1)
listings_filtered = listings_filtered.drop(columnas_propiedad[1:], axis=1)
listings_filtered = listings_filtered.drop(columnas_precio[1:], axis=1)
listings_filtered = listings_filtered.drop(columnas_disponibilidad[1:], axis=1)
listings_filtered = listings_filtered.drop(columnas_review[1:], axis=1)

Una Vez terminada la etapa de Extracción, pasamos a la etapa de procesamiento

En este caso comenzamos con el DataFrame **info1**.
Ejecutando el metodo ***info()***, vemos que hay 3 columnas con valores nulos y procedemos a quitarlas.

In [ ]:
# info1 = info1.drop(['thumbnail_url', 'medium_url', 'xl_picture_url'], axis=1).copy()

info1.info()

De esta manera seguimos trabajando con los siguientes frames

In [22]:
info2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23729 entries, 0 to 23728
Data columns (total 10 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            23729 non-null  int64  
 1   street                                        23729 non-null  object 
 2   neighbourhood_cleansed                        23729 non-null  object 
 3   neighbourhood_group_cleansed                  0 non-null      float64
 4   require_guest_profile_picture                 23729 non-null  object 
 5   require_guest_phone_verification              23729 non-null  object 
 6   calculated_host_listings_count                23729 non-null  int64  
 7   calculated_host_listings_count_entire_homes   23729 non-null  int64  
 8   calculated_host_listings_count_shared_rooms   23729 non-null  int64  
 9   calculated_host_listings_count_private_rooms  23729 non-null 

In [23]:
info2 = info2.drop(columns=['neighbourhood_group_cleansed'], axis=1).copy()

In [24]:
info2['require_guest_profile_picture'] = info2['require_guest_profile_picture'].apply(lambda x: True if x == 't' else False).copy()
info2['require_guest_phone_verification'] = info2['require_guest_phone_verification'].apply(lambda x: True if x == 't' else False).copy()

In [26]:
info2.iloc[:,4].value_counts()

require_guest_phone_verification
False    23411
True       318
Name: count, dtype: int64

In [50]:
location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23729 entries, 0 to 23728
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 23729 non-null  int64   
 1   neighbourhood      23729 non-null  object  
 2   city               23142 non-null  category
 3   state              23546 non-null  object  
 4   zipcode            18542 non-null  object  
 5   market             23708 non-null  object  
 6   smart_location     23729 non-null  object  
 7   country_code       23729 non-null  category
 8   country            23729 non-null  object  
 9   latitude           23729 non-null  float64 
 10  longitude          23729 non-null  float64 
 11  is_location_exact  23729 non-null  bool    
dtypes: bool(1), category(2), float64(2), int64(1), object(6)
memory usage: 1.8+ MB


In [ ]:
# location['is_location_exact'] = location['is_location_exact'].apply(lambda x: True if x == 't'else False).copy()
# location.country_code = location.country_code.astype('category')
# location.info()

# ciudad = location['city'].astype('category').copy()

# location.iloc [:,2] = ciudad

location['country'] = location['country'].astype('category').copy()

location.info()

In [ ]:
location['neighbourhood'] = location['neighbourhood'].astype('category').copy()

location.info()

In [283]:
# location[location.state.isin(['Autonomous City of Buenos Aires'])].state = 'CABA'
# location['state'] = location['state'].replace('-1', 'CABA')


# location.state.value_counts()

location.loc[:,'state'] = 'Buenos Aires'
location.loc[:,'city'] = 'Ciudad de Buenos Aires'

In [189]:
# location.loc[location['neighbourhood'] == 'Villa del Parque','state'] = 'CABA'

In [286]:

datos_propiedad.loc[:,'property_type'] = datos_propiedad['property_type'].astype('category').copy()
datos_propiedad.loc[:,'room_type'] = datos_propiedad['room_type'].astype('category').copy()
datos_propiedad.loc[:,'bed_type'] = datos_propiedad['bed_type'].astype('category').copy()


datos_propiedad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23729 entries, 0 to 23728
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   id             23729 non-null  int64   
 1   property_type  23729 non-null  category
 2   room_type      23729 non-null  category
 3   accommodates   23729 non-null  int64   
 4   bathrooms      23667 non-null  float64 
 5   bedrooms       23686 non-null  float64 
 6   beds           23562 non-null  float64 
 7   bed_type       23729 non-null  category
 8   amenities      23729 non-null  object  
 9   square_feet    383 non-null    float64 
dtypes: category(3), float64(4), int64(2), object(1)
memory usage: 1.3+ MB


In [287]:
precios

,id,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people
0,11508,3983.0,39828.0,82975.0,0.0,3319.0,1,0.0
1,12463,1593.0,NaN,NaN,NaN,NaN,1,0.0
2,13095,2987.0,NaN,NaN,NaN,NaN,1,0.0
3,13096,2987.0,NaN,NaN,19914.0,1328.0,1,0.0
4,13097,2987.0,NaN,NaN,NaN,NaN,1,996.0
...,...,...,...,...,...,...,...,...
23724,43278316,5111.0,NaN,NaN,NaN,NaN,1,0.0
23725,43280399,1328.0,NaN,NaN,NaN,NaN,1,0.0
23726,43280552,2390.0,NaN,NaN,NaN,NaN,1,0.0
23727,43281112,2854.0,NaN,NaN,NaN,NaN,1,0.0


In [288]:
# precios = listings[columnas_precio]

# nuevos_precios = precios['price'].str.replace('$','').str.replace(',','').astype('float')
# precios.iloc[:,1] = nuevos_precios

# precios['weekly_price'] = precios['weekly_price'].str.replace('$','').str.replace(',','').astype('float')
# precios['weekly_price'] = precios['weekly_price'].astype('float')

# precios['monthly_price'] = precios['monthly_price'].str.replace('$','').str.replace(',','').astype('float')
# precios['monthly_price'] = precios['monthly_price'].astype('float')

# precios.loc[:,'security_deposit'] = precios['security_deposit'].str.replace('$','').str.replace(',','').astype('float')
columna = 'extra_people'
# precios.loc[:, columna ] = precios[columna].str.replace('$','').str.replace(',','').astype('float')
# precios.iloc[:,1:3][precios['weekly_price'] > 0]

In [88]:
info1.to_sql(name="info1", con=cnx, if_exists="replace")
info2.to_sql(name='info2', con = cnx, if_exists="replace")
location.to_sql(name='location', con = cnx, if_exists="replace")
datos_propiedad.to_sql(name='datos_propiedad', con=cnx, if_exists='replace')
precios.to_sql(name='precios', con=cnx, if_exists='replace')
disponibilidad.to_sql(name="disponibilidad", con=cnx, if_exists='replace')
reviews_info.to_sql(name='reviews_info', con=cnx, if_exists='replace')



23729

Finalmente Guardamos la tabla listings

In [89]:
listings_filtered.to_sql(name="listings", con=cnx, if_exists='replace')

23729

In [73]:

reviews = pd.read_csv('../datasets/reviews.csv')

reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387099 entries, 0 to 387098
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     387099 non-null  int64 
 1   id             387099 non-null  int64 
 2   date           387099 non-null  object
 3   reviewer_id    387099 non-null  int64 
 4   reviewer_name  387099 non-null  object
 5   comments       386923 non-null  object
dtypes: int64(3), object(3)
memory usage: 17.7+ MB


In [74]:
reviewrs = reviews[['reviewer_id', 'reviewer_name']]

reviewrs.to_sql(name='reviewer', con=cnx, if_exists='replace')

387099

In [76]:
reviews_filtered = reviews.drop(['reviewer_name'], axis= 1)

reviews_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387099 entries, 0 to 387098
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   listing_id   387099 non-null  int64 
 1   id           387099 non-null  int64 
 2   date         387099 non-null  object
 3   reviewer_id  387099 non-null  int64 
 4   comments     386923 non-null  object
dtypes: int64(3), object(2)
memory usage: 14.8+ MB


In [77]:
reviews.to_sql(name='reviews', con=cnx, if_exists='replace')

387099

In [7]:
reviews.to_parquet('../datasets/reviews.parquet')

In [ ]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment import SentimentIntensityAnalyzer

# Create an instance of the sentiment intensity analyzer
sid = SentimentIntensityAnalyzer()

# Sample reviews
# comments = reviews.comments

# Analyze the sentiment of each review
for i, review in reviews.iterrows():
    # print(review)
    if i == 100:
        break
    try:
        scores = sid.polarity_scores(review.comments)
        compound = scores['compound']
        # for key in sorted(scores):
            # print(f"{key}: {scores[key]}", end=" ")
        # for key in sorted(scores):
        
        if compound > .4 and compound < .6:
            print(compound, review.listing_id, review.reviewer_name, review.comments, "\n", end=" ")
    except:
        pass
    
# sid.polarity_scores("Excellent, genius, best in the world")

In [6]:
# reviews = pd.read_csv('../datasets/reviews.csv')
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,11508,1615861,2012-07-02,877808,Charlie,Amazing place!\r\n\r\nLocation: short walk to ...
1,11508,3157005,2012-12-26,656077,Shaily,Really enjoyed Candela's recommendations and q...
2,11508,3281011,2013-01-05,2835998,Michiel,Candela and her colleague were very attentive ...
3,11508,6050019,2013-07-28,4600436,Tara,"The apartment was in a beautiful, modern build..."
4,11508,9328455,2013-12-22,3130017,Simon,My stay at Candela's apartment was very enjoya...
...,...,...,...,...,...,...
387094,42974156,621670219,2020-04-03,270233993,Carolina,Muchas gracias Mariano por la amabilidad en to...
387095,42975917,620648461,2020-03-23,342208450,Guillermo,"Me encanto el lugar. Impecable, moderno, y ate..."
387096,42990298,622364643,2020-04-13,342811096,Heber,"Lugar muy bien ubicado y tal cual las fotos, c..."
387097,43080350,622571105,2020-04-17,184553721,Elisabeth,"The apartment is a beautiful, small and good l..."


In [52]:
listings.iloc[:,-20]

0         95.0
1         95.0
2        100.0
3          NaN
4         99.0
         ...  
23724      NaN
23725      NaN
23726      NaN
23727      NaN
23728      NaN
Name: review_scores_rating, Length: 23729, dtype: float64

In [ ]:
import pandas as pd

pd.Da